In [1]:
%pip install pandas webdriver_manager selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 14.0 MB/s eta 0:00:00


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 123.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 128.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 117.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.9/507.9 kB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.8/346.8 kB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 16.0 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.4
    Uninstalling websocket-client-1.6.4:
      Successfully uninstalled websocket-client-1.6.4
  Attempting uninstall: typing_extensions
    Found existing 

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
chrome_options = Options()
# Important Arguments won't eun without them in Gitpod
chrome_options.add_argument("--disable-dev-shm-usage") 
chrome_options.add_argument("--headless")  

# Setup ChromeDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

In [4]:
def get_perfume_links(base_url, max_pages):
    perfume_links = []
    
    for page in range(1, max_pages + 1):
        url = f"{base_url}/page/{page}/"
        print(f"Scraping: {url}")
        driver.get(url)
        time.sleep(3)

        perfumes = driver.find_elements(By.CSS_SELECTOR, "h4.mkdf-product-list-title a")

        for perfume in perfumes:
            name = perfume.text
            link = perfume.get_attribute("href")
            perfume_links.append({"Name": name, "URL": link})

    return perfume_links

In [5]:
def get_perfume_details(url):
    driver.get(perfume["URL"])
    time.sleep(2)

    name = perfume["Name"]

    # Extract price
    try:
        price = driver.find_element(By.TAG_NAME, "bdi").text
    except:
        price = "Not Available"

    # Extract notes
    try:
        notes_element = driver.find_element(By.CLASS_NAME, "woocommerce-product-details__short-description")
        notes = notes_element.find_element(By.TAG_NAME, "p").get_attribute("innerHTML")
        notes_list = notes.split("<br>")
        

        top_notes = next((line.replace("Top notes:", "").strip() for line in notes_list if "Top notes:" in line), "Not Available")
        middle_notes = next((line.replace("Middle notes:", "").strip() for line in notes_list if "Middle notes:" in line), "Not Available")
        base_notes = next((line.replace("Base notes:", "").strip() for line in notes_list if "Base notes:" in line), "Not Available")

    except:
        try:
            top_notes_element = driver.find_element(By.XPATH, "//tr[contains(@class, 'attribute_pa_top-notes')]/td")
            top_notes = ", ".join([a.text.strip() for a in top_notes_element.find_elements(By.TAG_NAME, "a")])
        except:
            top_notes = "Not Available"

        try:
            middle_notes_element = driver.find_element(By.XPATH, "//tr[contains(@class, 'attribute_pa_middle-notes')]/td")
            middle_notes = ", ".join([a.text.strip() for a in middle_notes_element.find_elements(By.TAG_NAME, "a")])
        except:
            middle_notes = "Not Available"
        try:
            base_notes_element = driver.find_element(By.XPATH, "//tr[contains(@class, 'attribute_pa_base-notes')]/td")
            base_notes = ", ".join([a.text.strip() for a in base_notes_element.find_elements(By.TAG_NAME, "a")])
        except:
            base_notes = "Not Available"
    try:
        notes_element = driver.find_element(By.XPATH, "//*[@id='tab-description' or contains(@class, 'woocommerce-Tabs-panel') or contains(@class, 'woocommerce-product-details__short-description')]")
        notes = notes_element.find_elements(By.TAG_NAME, "p")

        notes_list= [p.get_attribute("innerHTML").strip() for p in notes]
    
        for line in notes_list:
            if "Top notes" in line:
                top_notes = re.sub(r"Top notes (are|:)", "", line).strip()
            elif "Middle notes" in line:
                middle_notes = re.sub(r"Middle notes (are|:)", "", line).strip()
            elif "Base notes" in line:
                base_notes = re.sub(r"Base notes (are|:)", "", line).strip()
    except:
        top_notes= middle_notes =base_notes = "Not Available"

    # Extract brand
    try:
        brand = driver.find_element(By.TAG_NAME, 'h3').text
    except:
        try:
            tab_element = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, 'a[aria-controls="tab-pwb_tab"]'))
            )
            tab_element.click()

            WebDriverWait(driver, 5).until(
            EC.text_to_be_present_in_element_attribute((By.CSS_SELECTOR, 'a[aria-controls="tab-pwb_tab"]'), "aria-selected", "true")
            )
            brand_name_element = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, '#tab-pwb_tab-content h3'))
            )
            brand = brand_name_element.text.strip()
        except:
            brand = "Unknown"

    # Extract reviews
    try:
        reviews_tab = driver.find_element(By.XPATH, '//a[@href="#tab-reviews"]')
        driver.execute_script("arguments[0].click();", reviews_tab)
        time.sleep(2)

        reviews_section = driver.find_element(By.ID, "tab-reviews")
        reviews = reviews_section.text.strip()

        if "There are no reviews yet." in reviews:
            reviews = "No Reviews Available"

    except Exception as e:
        print("Error extracting reviews:", e)
        reviews = "No Reviews Available"

    return {
        "name": name,
        "price": price,
        "brand": brand,
        "top_notes": top_notes,
        "middle_notes": middle_notes,
        "base_notes": base_notes,
        "reviews": reviews,
        "url": perfume["URL"]
    }


In [6]:
url = 'https://middleeasternscents.co.ke/product/batoul-12-ml/'
driver.get(url)
test = driver.find_element(By.XPATH, "//*[@id='tab-description' or contains(@class, 'woocommerce-Tabs-panel') or contains(@class, 'woocommerce-product-details__short-description')]")
test_notes = test.find_element(By.TAG_NAME, "p")

notes_ex = test_notes.get_attribute("innerHTML").split("<br>")
print(notes_ex)

['Fragrance note :']


In [8]:
# url = 'https://middleeasternscents.co.ke/product/miss-armaf-mystique/'
# driver.get(url)
# woocommerce-product-details__short-description

# print(notes_ex)

In [9]:
shop_url = "https://middleeasternscents.co.ke/shop/"
max_pages = 42
perfume_data = []

perfume_links = get_perfume_links(shop_url,max_pages)


Scraping: https://middleeasternscents.co.ke/shop//page/1/


Scraping: https://middleeasternscents.co.ke/shop//page/2/
Scraping: https://middleeasternscents.co.ke/shop//page/3/
Scraping: https://middleeasternscents.co.ke/shop//page/4/
Scraping: https://middleeasternscents.co.ke/shop//page/5/
Scraping: https://middleeasternscents.co.ke/shop//page/6/
Scraping: https://middleeasternscents.co.ke/shop//page/7/
Scraping: https://middleeasternscents.co.ke/shop//page/8/
Scraping: https://middleeasternscents.co.ke/shop//page/9/
Scraping: https://middleeasternscents.co.ke/shop//page/10/
Scraping: https://middleeasternscents.co.ke/shop//page/11/
Scraping: https://middleeasternscents.co.ke/shop//page/12/
Scraping: https://middleeasternscents.co.ke/shop//page/13/
Scraping: https://middleeasternscents.co.ke/shop//page/14/
Scraping: https://middleeasternscents.co.ke/shop//page/15/
Scraping: https://middleeasternscents.co.ke/shop//page/16/
Scraping: https://middleeasternscents.co.ke/shop//page/17/
Scraping: https://middleeasternscents.co.ke/shop//page/18/
Scrap

In [ ]:
for perfume in perfume_links:
    details = get_perfume_details(perfume)
    perfume_data.append(details)
    print(details)

{'name': '9AM DIVE', 'price': '4,500/=', 'brand': 'AFNAN', 'top_notes': 'Top notes: Mint, Lemon, Pink Pepper and Black Currant.<br>\nMiddle notes: Apple, Incense and Cedar.<br>\nBase notes: Patchouli, Ginger, Jasmine and Sandalwood.', 'middle_notes': 'Apple, Incense and Cedar.', 'base_notes': 'Patchouli, Ginger, Jasmine and Sandalwood.', 'reviews': 'No Reviews Available', 'url': 'https://middleeasternscents.co.ke/product/9am-dive/'}


{'name': '9PM', 'price': '4,500/=', 'brand': '', 'top_notes': 'Top notes: Apple, Cinnamon, Wild Lavender and Bergamot.<br>\nMiddle notes: Orange Blossom and Lily-of-the-Valley.<br>\nBase notes: Vanilla, Tonka Bean, Amber and Patchouli.', 'middle_notes': 'Not Available', 'base_notes': 'Not Available', 'reviews': 'No Reviews Available', 'url': 'https://middleeasternscents.co.ke/product/9pm/'}
{'name': 'ADEEB', 'price': '3,600/=', 'brand': '', 'top_notes': 'Top notes: Cinnamon, Vanilla, Tobacco.<br>\nMiddle notes: Moroccan spices, Herbs, Thyme, Clove, Smoky notes, Oud.<br>\nBase notes: Black amber, Patchouli, Cedarwood, Sandalwood, Musk.', 'middle_notes': 'Moroccan spices, Herbs, Thyme, Clove, Smoky notes, Oud.', 'base_notes': 'Black amber, Patchouli, Cedarwood, Sandalwood, Musk.', 'reviews': 'No Reviews Available', 'url': 'https://middleeasternscents.co.ke/product/adeeb/'}
{'name': 'ADWAA AL SHARQ', 'price': '2,300/=', 'brand': '', 'top_notes': 'Not Available', 'middle_notes': 'Not Avail

In [1]:
import pandas as pd
df = pd.DataFrame(perfume_data)
df.to_csv("perfumes.csv", index=False)

print("Scraping complete! Data saved in perfumes.csv")

driver.quit()

ModuleNotFoundError: No module named 'pandas'